# AIMUSIC

# Dependencies

In [152]:
print("[!]INFO: Importing dependencies ...")

# General
import numpy as np 
import pandas as pd
from glob import glob
import IPython
import pickle

# Data preprocessing
from music21 import converter, instrument, note, chord, stream
import music21

# Model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten
from keras.callbacks import ModelCheckpoint

[!]INFO: Importing dependencies ...


# Global Variables

In [153]:
print("[!]INFO: Setting up global variables ...")

DATASET = glob('data/dataset/*.mid')
WEIGHTS_COUNTER = 0
EPOCHS = 1
CHECKPOINT_WEIGHTS = 'weights/weights.dataset.'+ str(WEIGHTS_COUNTER) +'.hdf5'
GENERATED_MIDI = 'generated/' + str(WEIGHTS_COUNTER) + '.mid'
NUM_OF_NOTES_TO_BE_GENERATED = 500
INPUT_SEQUENCE_LENGTH = 100

# DATASET = DATASET[:2]

print()

[!]INFO: Setting up global variables ...



# Extracting Notes

In [154]:
def extract_notes():
    print("\t[!]INFO: Extracting notes from dataset.")
    
    try:
        notes = []
        for file in DATASET:
            # Converting .mid file to stream object/bytes
            midi = converter.parse(file)
            notes_to_parse = []

            try:
                # Given a single stream, partition into a part for each unique instrument
                parts = instrument.partitionByInstrument(midi)
            except:
                pass

            if parts: 
                notes_to_parse = parts.parts[0].recurse()
            else:
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse: 
                if isinstance(element, note.Note):
                    # If element is a note, extract pitch
                    notes.append(str(element.pitch))
                elif(isinstance(element, chord.Chord)):
                    # If element is a chord, append the normal form of the chord (a list of integers) to the list of notes
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        print("\t[+]SUCCESS: Successfully extracted notes.")
    except:
        print("\t[-]ERROR: Error in extracting notes.")
    
    # Saving the notes to a file
    print("\t[!]INFO: Saving notes to file.")
    
    try:
        with open('data/notes/notes', 'wb') as filepath:
            pickle.dump(notes, filepath)
        return notes
        print("\t[+]SUCCESS: Successfully written notes to file.")
    except:
        print("\t[-]ERROR: Error in writing notes to file.")

# Preparing Vocabulary

In [155]:
def prepare_sequence_vocab(notes, n_vocab): 
    print("\t[!]INFO: Preparing input sequences/vocabulary.")
    
    try:
        sequence_length = INPUT_SEQUENCE_LENGTH

        # Extract the unique pitches in the list of notes.
        pitchnames = sorted(set(item for item in notes))

        # Create a dictionary to map pitches to integers
        note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

        network_input = []
        network_output = []

        # Create input sequences and the corresponding outputs
        for i in range(0, len(notes) - sequence_length, 1):
            sequence_in = notes[i: i + sequence_length]
            sequence_out = notes[i + sequence_length]
            network_input.append([note_to_int[char] for char in sequence_in])
            network_output.append(note_to_int[sequence_out])

        n_patterns = len(network_input)

        # Reshape the input into a format comatible with LSTM layers 
        network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

        # Normalize input
        network_input = network_input / float(n_vocab)

        # One hot encode the output vectors
        network_output = np_utils.to_categorical(network_output)

        return network_input, network_output
        print("\t[+]SUCCESS: Successfully prepared vocabulary.")
    except:
        print("\t[-]ERROR: Error in preparing vocabulary.")

# Initializing Network

In [156]:
def init_network(network_in, n_vocab):
    print("\t[!]INFO: Initializing model.")
    
    try:
        model = Sequential()
        model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(128, return_sequences=True))
        model.add(Flatten())
        model.add(Dense(256))
        model.add(Dropout(0.3))
        model.add(Dense(n_vocab))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam')

        return model
        print("\t[+]SUCCESS: Successfully initialized model.")
    except:
        print("\t[-]ERROR: Error in initializing model.")

# Model Training

In [157]:
def train(model, network_input, network_output, epochs):
    print("\t[!]INFO: Model training in process.")
    
    try:
        # Create checkpoint to save the best model weights.
        checkpoint = ModelCheckpoint(CHECKPOINT_WEIGHTS, monitor='loss', verbose=0, save_best_only=True)

        model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])
        print("\t[+]SUCCESS: Successfully trained the model.")
    except:
        print("\t[-]ERROR: Error in training the model.")

In [158]:
def train_network():
    print("[!]INFO: Data-preprocessing in process ...")
    notes = extract_notes()
    n_vocab = len(set(notes))
    network_in, network_out = prepare_sequence_vocab(notes, n_vocab)
    print("[!]INFO: Data-preprocessing completed.\n")
    
    print("[!]INFO: Model training in process ...")
    model = init_network(network_in, n_vocab)
    train(model, network_in, network_out, EPOCHS)
    print("[!]INFO: Model training completed.\n")
    return model

In [159]:
train_network()

[!]INFO: Data-preprocessing in process ...
	[!]INFO: Extracting notes from dataset.
	[+]SUCCESS: Successfully extracted notes.
	[!]INFO: Saving notes to file.
	[!]INFO: Preparing input sequences/vocabulary.
[!]INFO: Data-preprocessing completed.

[!]INFO: Model training in process ...
	[!]INFO: Initializing model.
	[!]INFO: Model training in process.
Epoch 1/1
2843/2843 [==============================] - 35s - loss: 3.7954      

# Generating New MIDI files

In [160]:
def generate_midi():
    print("[!]INFO: Generating MIDI file in process ...")
    
    try:
        #Load the notes used to train the model
        with open('data/notes/notes', 'rb') as filepath:
            notes = pickle.load(filepath)

        # Get all pitch names
        pitchnames = sorted(set(item for item in notes))
        
        # Get all pitch names
        n_vocab = len(set(notes))

        network_input = get_input_sequences(notes, pitchnames, n_vocab)
        normalized_input = np.array(network_input) 
        normalized_input = np.reshape(normalized_input, (len(network_input), INPUT_SEQUENCE_LENGTH, 1))
        
        model = init_network(normalized_input, n_vocab)
        print('\t[!]INFO: Loading model ...')
        model.load_weights(CHECKPOINT_WEIGHTS)
        print('\t[+]SUCCESS: Model loaded successfully.')
        
        prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
        deextract_notes(prediction_output)
        
        print("\t[+]SUCCESS: Successfully generated MIDI file.")
    except:
        print("\t[-]ERROR: Error in generating MIDI file.")
    print("[!]INFO: MIDI file generation completed.\n")

In [161]:
def get_input_sequences(notes, pitchnames, n_vocab):
    # Map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    sequence_length = INPUT_SEQUENCE_LENGTH
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
    return (network_input)

In [166]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    # Pick a random integer
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # Pick a random sequence from the input as a starting point for the prediction
    pattern = network_input[start]
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    
    prediction_output = []
    
    print('\t[!]INFO: Generating notes.')

    try:
        # Generate NUM_OF_NOTES_TO_BE_GENERATED notes
        for note_index in range(NUM_OF_NOTES_TO_BE_GENERATED):
            prediction_input = np.reshape(pattern, (1, len(pattern), 1))
            prediction_input = prediction_input / float(n_vocab)

            prediction = model.predict(prediction_input, verbose=0)

            # Predicted output is the argmax(P(h|D))
            index = np.argmax(prediction)
            # Mapping the predicted interger back to the corresponding note
            result = int_to_note[index]
            # Storing the predicted output
            prediction_output.append(result)

            pattern.append(index)
            # Next input to the model
            pattern = pattern[1:len(pattern)]
        print("\t[+]SUCCESS: Successfully generated notes.")
        return prediction_output
    except:
        print("\t[-]ERROR: Error in generating notes.")

In [167]:
def deextract_notes(prediction_output):
    offset = 0
    output_notes = []

    # Create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Violin()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Violin()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=GENERATED_MIDI)

# Helper Functions

In [168]:
def get_int_to_note():
    #Load the notes used to train the model
    with open('data/notes/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))    
    return dict((number, note) for number, note in enumerate(pitchnames))

def get_note_to_int():
    #Load the notes used to train the model
    with open('data/notes/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))    
    return dict((note, number) for number, note in enumerate(pitchnames))
    
def get_vocabulary():
    #Load the notes used to train the model
    with open('data/notes/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    return sorted(set(item for item in notes))

In [169]:
generate_midi()

[!]INFO: Generating MIDI file in process ...
	[!]INFO: Initializing model.
	[!]INFO: Loading model ...
	[+]SUCCESS: Model loaded successfully.
Pattern: 50
	[!]INFO: Generating notes.
	[+]SUCCESS: Successfully generated notes.
	[+]SUCCESS: Successfully generated MIDI file.
[!]INFO: MIDI file generation completed.



In [151]:
len(get_vocabulary())

82

In [ ]:
## Ask user to enter unique notes for the first time
## Ask user to enter the length of notes so that audio may be generated of that length